In [1]:
import sklearn

#NN Surrogate model class
from injector_surrogate_quads import *

sys.path.append('../configs')
#Sim reference point to optimize around
from ref_config import ref_point

#BO
from bayes_opt import BayesianOptimization

# Load injector model

In [2]:
Model = Surrogate_NN()

Model.load_saved_model(model_path = '../models/', \
                       model_name = 'model_OTR2_NA_rms_emit_elu_2021-07-27T19_54_57-07_00')
Model.load_scaling()
Model.take_log_out = False

# Set up sampling and objectives

In [3]:
#convert to machine units
ref_point = Model.sim_to_machine(np.asarray(ref_point))

#input params: solenoid and quads to vary 
opt_var_names = ['SOL1:solenoid_field_scale', 'SQ01:b1_gradient']#,'CQ01:b1_gradient'] 

#output params: emittance in transverse plane (x & y)
opt_out_names = ['norm_emit_x','norm_emit_y']

def evaluate(varx,vary):#,varz): 

    #make input array of length model_in_list (inputs model takes)
    x_in = np.empty((1,len(Model.model_in_list)))

    #fill in reference point around which to optimize
    x_in[:,:] = np.asarray(ref_point[0])

    #set solenoid, SQ, CQ to values from optimization step
    x_in[:, Model.loc_in[opt_var_names[0]]] = varx
    x_in[:, Model.loc_in[opt_var_names[1]]] = vary
    #x_in[:, Model.loc_in[opt_var_names[2]]] = varz

    #output predictions
    y_out = Model.pred_machine_units(x_in) 

    return -1*objective(y_out)[0]


def objective(y_out):
    
    #output is geometric emittance in transverse plane
    out1 = y_out[:,Model.loc_out['norm_emit_x']] #grab norm_emit_x out of the model
    out2 = y_out[:,Model.loc_out['norm_emit_y']] #grab norm_emit_y out of the model
       
    return np.sqrt(out1*out2)/1e-6 # in um units

# Simple BO

In [4]:
# bounds on input params 
pbounds = {'varx': (0.43, 0.56),
           'vary': (-0.025, 0.025),
           #'varz': (-0.02, 0.02)
          }

optimizer = BayesianOptimization(
    f = evaluate,
    pbounds = pbounds,
    random_state = 1,
)

optimizer.maximize(
    init_points=5,
    n_iter=40,
)

|   iter    |  target   |   varx    |   vary    |
-------------------------------------------------
|  1        | -0.7367   |  0.4842   |  0.01102  |
|  2        | -2.08     |  0.43     | -0.009883 |
|  3        | -1.902    |  0.4491   | -0.02038  |
|  4        | -1.423    |  0.4542   | -0.007722 |
|  5        | -0.59     |  0.4816   |  0.001941 |
|  6        | -3.108    |  0.5131   | -0.01694  |
|  7        | -0.5999   |  0.482    |  0.000813 |
|  8        | -6.045    |  0.56     |  0.025    |
|  9        | -2.423    |  0.43     |  0.025    |
|  10       | -1.145    |  0.4688   |  0.025    |
|  11       | -0.6113   |  0.4745   | -0.005768 |
|  12       | -0.6383   |  0.4725   |  0.005646 |
|  13       | -1.07     |  0.4797   | -0.025    |
|  14       | -2.089    |  0.5019   |  0.0101   |
|  15       | -0.7401   |  0.4843   |  0.01097  |
|  16       | -0.5617   |  0.4771   |  0.000200 |
|  17       | -0.565    |  0.4777   |  0.005558 |
|  18       | -0.5733   |  0.4787   | -0.003802 |


## Results from simple BO

In [7]:
SOL_opt = optimizer.max['params']['varx'] # solenoid val at optimum
CQ_opt = optimizer.max['params']['vary'] # CQ val at optimum
SQ_opt = optimizer.max['params']['varz'] # SQ val at optimum

opt_emit = -1*optimizer.max['target'] # emittance value at optimum (in um)

print('optimum (pv_units) ',SOL_opt, CQ_opt, SQ_opt)
print('optimum geom emit ', opt_emit)

optimum (pv_units)  0.47737014691426854 -0.0004981337721706551 0.0004994750148957644
optimum geom emit  0.5750189423561096
